# 1　初期設定（毎回実行）

In [1]:
# SentencePieceのインストール
!pip install sentencepiece
!pip install datasets

# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
import torch
import random
from datasets import load_dataset, Dataset
from tokenizers import BertWordPieceTokenizer
from transformers import (
    BertConfig,
    BertForMaskedLM,
    BertTokenizerFast,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
os.chdir('/content/drive/MyDrive/metamorphosis')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whic

# ２: 関数の定義（次の指示が出てくるまで全部実行）

In [2]:
def func_load_codes_from_codesearchnet(num_samples: int = 10000):
    """
    CodeSearchNet の Python コードデータ（func_code_string）を抽出してリストに格納します。

    Args:
        num_samples (int): 抽出するサンプル数。0 以下（例: -1）を指定すると全サンプルを使用します。

    Returns:
        List[str]: 抽出したコードのリスト
    """
    dataset = load_dataset("code_search_net", "python", split="train")
    code_samples = []
    for item in dataset:
        code = item.get("func_code_string", "")
        # 空文字でなければ追加
        if code and len(code.strip()) > 0:
            code_samples.append(code)
        # num_samples が正の値の場合のみ打ち切り
        if num_samples > 0 and len(code_samples) >= num_samples:
            break
    print(f"抽出したコードサンプル数: {len(code_samples)}")
    return code_samples


In [3]:
def func_train_wordpiece_tokenizer(corpus, vocab_size: int = 32000, save_dir: str = "./codebert_tokenizer"):
    """
    CodeSearchNet のコードコーパスから WordPiece ベースのトークナイザーを学習し、保存する関数。

    Args:
        corpus (List[str]): コードの文字列リスト
        vocab_size (int): 語彙数（例: 32000）
        save_dir (str): トークナイザーの学習結果を保存するディレクトリ

    Returns:
        BertTokenizerFast: 学習済みの Transformers 用トークナイザー
    """
    os.makedirs(save_dir, exist_ok=True)
    temp_corpus_file = os.path.join(save_dir, "corpus.txt")

    # コーパスを一時ファイルに書き出す
    with open(temp_corpus_file, "w", encoding="utf-8") as f:
        for line in corpus:
            f.write(line.strip() + "\n")

    # BertWordPieceTokenizer の初期化（大文字小文字の区別は用途に合わせて調整）
    tokenizer = BertWordPieceTokenizer(lowercase=False)
    # 学習
    tokenizer.train(
        files=[temp_corpus_file],
        vocab_size=vocab_size,
        min_frequency=2,
        special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    )

    # 保存（vocab.txt などの形で保存される）
    tokenizer.save_model(save_dir)
    print(f"WordPiece トークナイザーを保存しました: {save_dir}")

    # Transformers 用ラッパーの作成
    transformers_tokenizer = BertTokenizerFast.from_pretrained(save_dir)
    return transformers_tokenizer


In [4]:

def func_create_bert_model(tokenizer, hidden_size: int = 768, num_hidden_layers: int = 12,
                           num_attention_heads: int = 12, intermediate_size: int = 512):
    """
    学習済みトークナイザーの語彙サイズに合わせた BERT (Masked LM 用) をゼロから初期化する関数。

    Args:
        tokenizer (BertTokenizerFast): 学習済みトークナイザー
        hidden_size (int): 隠れ層のサイズ
        num_hidden_layers (int): Transformer の層数
        num_attention_heads (int): アテンションヘッド数
        intermediate_size (int): Feed-forward の中間層サイズ

    Returns:
        BertForMaskedLM: 初期化された BERT モデル
    """
    vocab_size = tokenizer.vocab_size
    config = BertConfig(
        vocab_size=vocab_size,
        hidden_size=768,            # 改善：より大きな隠れ層
        num_hidden_layers=12,       # 改善：層数 12
        num_attention_heads=12,     # 改善：ヘッド数 12
        intermediate_size=3072,     # 改善：中間層サイズ 3072
        max_position_embeddings=512,
        type_vocab_size=2,
        hidden_dropout_prob=0.1,
        attention_probs_dropout_prob=0.1,
    )
    model = BertForMaskedLM(config)
    print(f"BERT モデルを初期化しました（語彙サイズ: {vocab_size}）")
    return model


In [5]:
import os
import re
from transformers import BertTokenizerFast, BertForMaskedLM

def get_latest_checkpoint(output_dir: str) -> str:
    """
    出力ディレクトリ内の 'checkpoint-*' というサブディレクトリのうち、
    最新のもの（チェックポイント番号が最も大きいもの）のパスを返す。
    存在しなければ None を返す。
    """
    checkpoint_dirs = []
    for subdir in os.listdir(output_dir):
        full_path = os.path.join(output_dir, subdir)
        if os.path.isdir(full_path) and subdir.startswith("checkpoint-"):
            m = re.search(r"checkpoint-(\d+)", subdir)
            if m:
                checkpoint_dirs.append((int(m.group(1)), full_path))
    if not checkpoint_dirs:
        return None
    # 数値順にソートして最新のものを返す
    checkpoint_dirs = sorted(checkpoint_dirs, key=lambda x: x[0])
    return checkpoint_dirs[-1][1]

def func_create_codebert_model(num_code_samples: int = 10000,
                               vocab_size: int = 32000,
                               tokenizer_save_dir: str = "./codebert_tokenizer",
                               model_checkpoint_dir: str = "./code_bert_output"):
    """
    CodeSearchNet のコードデータから WordPiece トークナイザーを学習し、
    その語彙サイズに合わせた BERT モデル（Masked LM 用）をゼロから作成する関数。

    ※ カレントディレクトリ内に `tokenizer_save_dir` が存在する場合は既存のトークナイザーを読み込み、
        また `model_checkpoint_dir` にチェックポイントがあればそれを読み込みます。

    Args:
        num_code_samples (int): トークナイザー学習に使用するコードサンプル数。全件使用する場合は 0 以下を指定。
        vocab_size (int): 語彙数（例: 32000）
        tokenizer_save_dir (str): トークナイザーの保存ディレクトリ
        model_checkpoint_dir (str): モデルのチェックポイント保存ディレクトリ

    Returns:
        tuple: (BertTokenizerFast, BertForMaskedLM)
    """
    # --- トークナイザーのロードまたは学習 ---
    if os.path.exists(tokenizer_save_dir):
        print(f"既存のトークナイザーを {tokenizer_save_dir} から読み込みます。")
        tokenizer = BertTokenizerFast.from_pretrained(tokenizer_save_dir)
    else:
        print("トークナイザーが存在しないため、新規に学習します。")
        # CodeSearchNet からコードを抽出
        code_corpus = func_load_codes_from_codesearchnet(num_samples=num_code_samples)
        # WordPiece トークナイザーの学習（内部で tokenizer_save_dir へ保存する処理を含む）
        tokenizer = func_train_wordpiece_tokenizer(code_corpus, vocab_size=vocab_size, save_dir=tokenizer_save_dir)

    # --- モデルのロードまたは新規作成 ---
    checkpoint_to_load = None
    if os.path.exists(model_checkpoint_dir):
        # まず、出力ディレクトリ内の最新チェックポイントを取得
        latest_checkpoint = get_latest_checkpoint(model_checkpoint_dir)
        if latest_checkpoint:
            checkpoint_to_load = latest_checkpoint
        # もしサブディレクトリが見つからなければ、model_checkpoint_dir直下のファイルをチェック
        elif any(os.path.exists(os.path.join(model_checkpoint_dir, fname))
                 for fname in ["pytorch_model.bin", "model.safetensors", "tf_model.h5",
                               "model.ckpt.index", "flax_model.msgpack"]):
            checkpoint_to_load = model_checkpoint_dir

    if checkpoint_to_load:
        print(f"チェックポイントが存在するため、モデルを {checkpoint_to_load} から読み込みます。")
        model = BertForMaskedLM.from_pretrained(checkpoint_to_load)
    else:
        print("チェックポイントが存在しないため、新規にモデルを作成します。")
        model = func_create_bert_model(tokenizer)

    return tokenizer, model


In [6]:
def func_tokenize_dataset(dataset, tokenizer, max_length: int = 512):
    """
    CodeSearchNet のデータセット（func_code_string）を tokenizer を用いてトークナイズします。

    Args:
        dataset (datasets.Dataset): load_dataset で取得したデータセット
        tokenizer (BertTokenizerFast): 学習済みトークナイザー
        max_length (int): トークナイズ時の最大シーケンス 長

    Returns:
        datasets.Dataset: トークナイズ済みのデータセット
    """
    def tokenize_function(example):
        texts = example["func_code_string"]
        # 各サンプルが None または空文字なら "" に置き換える
        texts = [text if text is not None and text.strip() != "" else "" for text in texts]
        # batched なのでリストをまとめて処理（返り値は dict になります）
        return tokenizer(texts, truncation=True, max_length=max_length)

    # func_code_string 以外のカラムは削除
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=[col for col in dataset.column_names if col != "func_code_string"]
    )
    return tokenized_dataset


In [7]:
def func_train_bert_model(model, tokenizer, train_dataset, output_dir: str = "./codebert_model",
                          num_train_epochs: int = 5, per_device_train_batch_size: int = 8):
    """
    Trainer を用いて、Masked LM 用に BERT を訓練する関数です。

    Args:
        model (BertForMaskedLM): BERT モデル
        tokenizer (BertTokenizerFast): 学習済みトークナイザー
        train_dataset (datasets.Dataset): トークナイズ済み訓練データセット
        output_dir (str): 訓練済みモデルの保存ディレクトリ
        num_train_epochs (int): 訓練エポック数
        per_device_train_batch_size (int): バッチサイズ

    Returns:
        Trainer: 訓練後の Trainer インスタンス
    """
    # DataCollator（ランダムマスク処理を自動実施）
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        save_steps=5000,
        save_total_limit=2,
        logging_steps=100,
        prediction_loss_only=True,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    print("訓練を開始します...")
    trainer.train()
    print("訓練が完了しました。")

    # 訓練済みモデルの保存
    trainer.save_model(output_dir)
    print(f"訓練済みモデルを保存しました: {output_dir}")
    return trainer


# ３:モデルの読み込み
##(トレーニングする際、トレーニングしたものを読み込む際は実行)

In [8]:
# --- モデル・トークナイザーの初期化 ---
# 例として、num_code_samples に -1 を指定すると全サンプル（可能な限り）を使用します
tokenizer, model = func_create_codebert_model(
    num_code_samples=-1,   # 全件使用する場合は -1 または 0 以下を指定
    vocab_size=32000,
    tokenizer_save_dir="./codebert_tokenizer",
    model_checkpoint_dir = "./code_bert_output"
)

# サンプルコードのトークナイズ例
sample_code = (
    "def add(a, b):\n"
    "    return a + b\n\n"
    "result = add(3, 5)\n"
    "print(result)"
)
encoded = tokenizer(sample_code, return_tensors="pt")
print("トークナイズ結果（input_ids）:")
print(encoded["input_ids"])

print("\nBERT モデルの構造:")
print(model)


既存のトークナイザーを ./codebert_tokenizer から読み込みます。


BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


チェックポイントが存在するため、モデルを ./code_bert_output/checkpoint-515225 から読み込みます。
トークナイズ結果（input_ids）:
tensor([[   2, 1358, 1497,   12,   69,   16,   70,   13,   30, 1352,   69,   15,
           70, 1528,   33, 1497,   12,   23,   16,   25,   13, 1701,   12, 1528,
           13,    3]])

BERT モデルの構造:
BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_fea

# 4: データセットの読み込み (基本的には実行）

In [9]:
import os
import re
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# --- 訓練用データセットの準備 ---
# CodeSearchNet の全訓練データを使用
raw_train_dataset = load_dataset("code_search_net", "python", split="train")
tokenized_train_dataset = func_tokenize_dataset(raw_train_dataset, tokenizer, max_length=512)

# --- 評価用データセットの準備 ---
# 例として validation split を使用（存在しない場合は test など適宜変更してください）
raw_eval_dataset = load_dataset("code_search_net", "python", split="validation")
tokenized_eval_dataset = func_tokenize_dataset(raw_eval_dataset, tokenizer, max_length=512)

# --- DataCollator の設定（ランダムマスクを自動実施） ---
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

code_search_net.py:   0%|          | 0.00/8.44k [00:00<?, ?B/s]

The repository for code_search_net contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/code_search_net.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


python.zip:   0%|          | 0.00/941M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/412178 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/22176 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/23107 [00:00<?, ? examples/s]

Map:   0%|          | 0/412178 [00:00<?, ? examples/s]

Map:   0%|          | 0/23107 [00:00<?, ? examples/s]

# 5　トレーニング関数　トレーニングする際は実行　それ以外は無視（実行に数日かかります！！）

In [ ]:
# --- Transformers Trainer の設定 ---
training_args = TrainingArguments(
    output_dir="./code_bert_output",
    run_name="code_bert_mlm_training",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    save_steps=10000,
    save_total_limit=2,
    prediction_loss_only=True,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    eval_strategy="steps",  # 例: "steps" または "epoch"
    eval_steps=500          # 評価を行うステップ数（"steps" の場合）
)

# 評価用データセットから最初の 100 個のサンプルだけを使用する
small_eval_dataset = tokenized_eval_dataset.select(range(400))

# Trainer のインスタンス化（評価用データセットに small_eval_dataset を渡す）
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,   # 訓練用データセット
    eval_dataset=small_eval_dataset,           # 評価用データセット（サブセット）
    data_collator=data_collator
)

# --- 出力ディレクトリ内の最新チェックポイントを取得する関数 ---
def get_latest_checkpoint(output_dir: str) -> str:
    """
    出力ディレクトリ内の 'checkpoint-*' というサブディレクトリのうち、
    最新のもの（チェックポイント番号が最も大きいもの）のパスを返す。
    存在しなければ None を返す。
    """
    checkpoint_dirs = []
    if not os.path.exists(output_dir):
        return None
    for subdir in os.listdir(output_dir):
        full_path = os.path.join(output_dir, subdir)
        if os.path.isdir(full_path) and subdir.startswith("checkpoint-"):
            m = re.search(r"checkpoint-(\d+)", subdir)
            if m:
                checkpoint_dirs.append((int(m.group(1)), full_path))
    if not checkpoint_dirs:
        return None
    # 数値順にソートして最新のチェックポイントのパスを返す
    checkpoint_dirs = sorted(checkpoint_dirs, key=lambda x: x[0])
    return checkpoint_dirs[-1][1]

# --- 最新のチェックポイントが存在するか確認し、あればそこから学習を再開 ---
latest_checkpoint = get_latest_checkpoint(training_args.output_dir)
if latest_checkpoint:
    print(f"チェックポイントが存在するため、{latest_checkpoint} から学習を再開します。")
    trainer.train(resume_from_checkpoint=latest_checkpoint)
else:
    print("チェックポイントが存在しないため、新規に学習を開始します。")
    trainer.train()

チェックポイントが存在するため、./code_bert_output/checkpoint-270000 から学習を再開します。


There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].
/usr/local/lib/python3.11/dist-packages/transformers/trainer.py:3441: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issu

Step,Training Loss,Validation Loss
270500,1.225600,1.284552
271000,1.229900,1.280250
271500,1.155400,1.285206
272000,1.175600,1.247794
272500,1.186400,1.287470
273000,1.217100,1.258223
273500,1.167200,1.303954
274000,1.183900,1.270468
274500,1.260800,1.256846
275000,1.238400,1.256587


Step,Training Loss,Validation Loss
270500,1.225600,1.284552
271000,1.229900,1.280250
271500,1.155400,1.285206
272000,1.175600,1.247794
272500,1.186400,1.287470
273000,1.217100,1.258223
273500,1.167200,1.303954
274000,1.183900,1.270468
274500,1.260800,1.256846
275000,1.238400,1.256587


In [26]:
# CodeBERT（WordPieceベース）のモデルとトークナイザーを Google Drive に保存
save_path_codebert = "/content/drive/MyDrive/codebert/"
os.makedirs(save_path_codebert, exist_ok=True)
model.save_pretrained(save_path_codebert)
tokenizer.save_pretrained(save_path_codebert)
print(f"✅ CodeBERTモデルと WordPiece トークナイザーを {save_path_codebert} に保存しました。")

✅ CodeBERTモデルと WordPiece トークナイザーを /content/drive/MyDrive/codebert/ に保存しました。


#6: モデルのテスト  モデルの動作を確認する用（単純なマスクタスクと関数名を予測するタスク）

In [27]:
import os
from transformers import BertForMaskedLM, BertTokenizerFast

def load_codebert_model_and_tokenizer(model_dir: str, tokenizer_dir: str = None):
    """
    保存済みの CodeBERT モデルとトークナイザーを読み込みます。

    Args:
        model_dir (str): 保存済みモデル（BertForMaskedLM）のディレクトリパス。
        tokenizer_dir (str, optional): 保存済みトークナイザー（BertTokenizerFast）のディレクトリパス。
            指定しない場合は model_dir と同じディレクトリから読み込みます。

    Returns:
        tuple: (tokenizer, model)
    """
    # tokenizer_dir が指定されなければ model_dir を使用
    if tokenizer_dir is None:
        tokenizer_dir = model_dir

    if not os.path.exists(model_dir):
        raise ValueError(f"指定されたモデルディレクトリが存在しません: {model_dir}")
    if not os.path.exists(tokenizer_dir):
        raise ValueError(f"指定されたトークナイザーディレクトリが存在しません: {tokenizer_dir}")

    print(f"モデルを {model_dir} から読み込みます。")
    model = BertForMaskedLM.from_pretrained(model_dir)

    print(f"トークナイザーを {tokenizer_dir} から読み込みます。")
    tokenizer = BertTokenizerFast.from_pretrained(tokenizer_dir)

    return tokenizer, model

# 例: Google Drive に保存したディレクトリから読み込む場合
drive_model_dir = "/content/drive/MyDrive/codebert"  # ※両方同じ場所の場合は省略可能
# もしトークナイザーは別ディレクトリに保存している場合は drive_tokenizer_dir にそのパスを指定する
drive_tokenizer_dir = "/content/drive/MyDrive/codebert"  # ※両方同じ場所の場合は省略可能

tokenizer, model = load_codebert_model_and_tokenizer(drive_model_dir, drive_tokenizer_dir)

# 読み込み確認のため、サンプルコードをトークナイズしてみます
sample_code = (
    "def add(a, b):\n"
    "    return a + b\n\n"
    "result = add(3, 5)\n"
    "print(result)"
)
encoded = tokenizer(sample_code, return_tensors="pt")
print("トークナイズ結果（input_ids）:")
print(encoded["input_ids"])

print("\n読み込んだ CodeBERT モデルの構造:")
print(model)


モデルを /content/drive/MyDrive/codebert から読み込みます。
トークナイザーを /content/drive/MyDrive/codebert から読み込みます。
トークナイズ結果（input_ids）:
tensor([[   2, 1358, 1497,   12,   69,   16,   70,   13,   30, 1352,   69,   15,
           70, 1528,   33, 1497,   12,   23,   16,   25,   13, 1701,   12, 1528,
           13,    3]])

読み込んだ CodeBERT モデルの構造:
BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=Tr

In [28]:
import os
import random
import torch
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizerFast


# --- CodeBERT を用いた MASK 付与・予測関数 ---
def mask_and_predict_codebert(text, top_k=5):
    """
    入力テキストを CodeBERT のトークナイザーでトークナイズし、
    先頭([CLS])・末尾([SEP])を除いた部分からランダムに1箇所を [MASK] に置換して、
    BERT により予測された単語（上位 top_k 件）を出力します。

    また、どこがマスクされたかが分かるように、トークン列も表示します。

    Args:
        text (str): 入力テキスト
        top_k (int): 予測結果として表示する上位件数
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # [CLS] と [SEP] を自動付与してエンコード
    token_ids = tokenizer.encode(text, add_special_tokens=True)
    if len(token_ids) <= 2:
        print("入力テキストが短すぎます:", text)
        return

    # [CLS]（先頭）と [SEP]（末尾）を除いた範囲でランダムにマスク位置を選択
    mask_index = random.randint(1, len(token_ids)-2)
    # マスク前のトークン列を取得（デバッグ用）
    original_tokens = tokenizer.convert_ids_to_tokens(token_ids)
    # 指定位置を [MASK] に置換
    token_ids[mask_index] = tokenizer.convert_tokens_to_ids("[MASK]")

    # マスク後のトークン列（どこがマスクされたか分かるようにハイライト）
    masked_tokens = []
    for idx, token in enumerate(tokenizer.convert_ids_to_tokens(token_ids)):
        if idx == mask_index:
            # ハイライト用に大文字で [MASK] と表示（必要に応じて色付けなども可能）
            masked_tokens.append("**[MASK]**")
        else:
            masked_tokens.append(token)
    masked_text = " ".join(masked_tokens)

    # テンソルに変換してデバイスに配置
    input_tensor = torch.tensor([token_ids], dtype=torch.long).to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)
        logits = outputs.logits  # shape: (1, seq_len, vocab_size)

    # マスク位置のロジットから上位 top_k 件を取得
    mask_logits = logits[0, mask_index]
    top = torch.topk(F.softmax(mask_logits, dim=-1), top_k)

    print("\n=== CodeBERT を用いた予測 ===")
    print(f"入力テキスト: {text}")
    print(f"マスク位置: {mask_index} 番目のトークン")
    print("トークン列（マスク箇所ハイライト）:")
    print(masked_text)
    print("予測された単語:")
    for i in range(top_k):
        pred_token = tokenizer.convert_ids_to_tokens(top.indices[i].item())
        prob = top.values[i].item()
        print(f"  {i+1}. {pred_token} ({prob:.2%})")

# 複数の例でテスト実行
sample_texts = [
    "def multiply(a, b):\n    return a * b",
    "if x > 10:\n    print('x is greater than 10')",
    "for i in range(5):\n    print(i)",
    "class Person:\n    def __init__(self, name):\n        self.name = name",
    "try:\n    risky_operation()\nexcept Exception as e:\n    print(e)"
]

for text in sample_texts:
    mask_and_predict_codebert(text, top_k=5)



=== CodeBERT を用いた予測 ===
入力テキスト: def multiply(a, b):
    return a * b
マスク位置: 8 番目のトークン
トークン列（マスク箇所ハイライト）:
[CLS] def multiply ( a , b ) **[MASK]** return a * b [SEP]
予測された単語:
  1. : (99.96%)
  2. . (0.01%)
  3. , (0.01%)
  4. @ (0.00%)
  5. ; (0.00%)

=== CodeBERT を用いた予測 ===
入力テキスト: if x > 10:
    print('x is greater than 10')
マスク位置: 12 番目のトークン
トークン列（マスク箇所ハイライト）:
[CLS] if x > 10 : print ( ' x is greater **[MASK]** 10 ' ) [SEP]
予測された単語:
  1. than (98.06%)
  2. to (0.61%)
  3. > (0.35%)
  4. or (0.26%)
  5. and (0.16%)

=== CodeBERT を用いた予測 ===
入力テキスト: for i in range(5):
    print(i)
マスク位置: 10 番目のトークン
トークン列（マスク箇所ハイライト）:
[CLS] for i in range ( 5 ) : print **[MASK]** i ) [SEP]
予測された単語:
  1. ( (100.00%)
  2. print (0.00%)
  3. _ (0.00%)
  4. i (0.00%)
  5. ##s (0.00%)

=== CodeBERT を用いた予測 ===
入力テキスト: class Person:
    def __init__(self, name):
        self.name = name
マスク位置: 6 番目のトークン
トークン列（マスク箇所ハイライト）:
[CLS] class person : def _ **[MASK]** init _ _ ( self , name ) : self . name = name [SEP]


In [16]:
import random
import torch
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizerFast

# --- valid データセットに対して MASK 付与・予測を適用する関数 ---
def evaluate_on_valid_dataset(eval_dataset, tokenizer, model, num_samples=5, top_k=5):
    """
    Validデータセットの一部のサンプルに対して CodeBERT を用いた MASK 付与・予測を実施し、
    その結果を表示する。

    Args:
        eval_dataset (Dataset): 評価用データセット（tokenized_eval_dataset）
        tokenizer (BertTokenizerFast): CodeBERTのトークナイザー
        model (BertForMaskedLM): CodeBERTのモデル
        num_samples (int): 実行するサンプルの個数（デフォルトは5）
        top_k (int): 予測結果として表示する上位件数
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # データセットからランダムに num_samples 件のサンプルを取得
    sampled_indices = random.sample(range(len(eval_dataset)), num_samples)

    for idx in sampled_indices:
        # サンプルのテキスト取得
        text = eval_dataset[idx]["func_code_string"]

        # トークン化（最大512トークンに制限）
        token_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)

        if len(token_ids) <= 2:
            print(f"入力テキストが短すぎるためスキップ: {text}")
            continue

        # マスク位置の決定（[CLS] と [SEP] を除いた範囲でランダムに選択）
        mask_index = random.randint(1, len(token_ids) - 2)
        original_tokens = tokenizer.convert_ids_to_tokens(token_ids)
        token_ids[mask_index] = tokenizer.convert_tokens_to_ids("[MASK]")

        # マスク後のトークン列の表示用リスト作成（ハイライト）
        masked_tokens = [
            "**[MASK]**" if i == mask_index else token
            for i, token in enumerate(tokenizer.convert_ids_to_tokens(token_ids))
        ]
        masked_text = " ".join(masked_tokens)

        # テンソル変換してモデルに入力
        # 修正後（既にリストで包んでいるのでバッチ次元は付いている）
        input_tensor = torch.tensor([token_ids], dtype=torch.long).to(device)

        model.eval()
        with torch.no_grad():
            outputs = model(input_tensor)
            logits = outputs.logits  # shape: (1, seq_len, vocab_size)

        # マスク位置のロジットから上位 top_k 件を取得
        mask_logits = logits[0, mask_index]
        top_predictions = torch.topk(F.softmax(mask_logits, dim=-1), top_k)

        print("\n=== CodeBERT を用いた予測 ===")
        print(f"入力テキスト (元データセット):\n{text[:300]}...")  # 長すぎる場合は最初の300文字のみ表示
        print(f"マスク位置: {mask_index} 番目のトークン")
        print("トークン列（マスク箇所ハイライト）:")
        print(masked_text)
        print("予測された単語:")
        for i in range(top_k):
            pred_token = tokenizer.convert_ids_to_tokens(top_predictions.indices[i].item())
            prob = top_predictions.values[i].item()
            print(f"  {i+1}. {pred_token} ({prob:.2%})")

# --- valid データセットを用いて予測を実行 ---
evaluate_on_valid_dataset(raw_eval_dataset, tokenizer, model, num_samples=5, top_k=5)



=== CodeBERT を用いた予測 ===
入力テキスト (元データセット):
def get_room_jid(self,nick=None):
        """
        Get own room JID or a room JID for given `nick`.

        :Parameters:
            - `nick`: a nick for which the room JID is requested.
        :Types:
            - `nick`: `unicode`

        :return: the room JID.
        :returntype: `JID`
  ...
マスク位置: 100 番目のトークン
トークン列（マスク箇所ハイライト）:
[CLS] def get _ room _ jid ( self , nick = none ) : " " " get own room jid or a room jid for given ` nick ` . : parameters : - ` nick ` : a nick for which the room jid is requested . : types : - ` nick ` : ` unicode ` : return : the room jid . : return ##type : ` jid ` " " " if nick is none : return self . room _ jid return jid ( self . room _ jid . node , **[MASK]** . room _ jid . domain , nick ) [SEP]
予測された単語:
  1. self (99.77%)
  2. nick (0.02%)
  3. node (0.01%)
  4. api (0.01%)
  5. constants (0.01%)

=== CodeBERT を用いた予測 ===
入力テキスト (元データセット):
def register(self, task_json=None, json_filename=None):
     

In [18]:
import re
import random

def mask_function_name_with_tokenizer(example, tokenizer, mask_all=False):
    """
    関数定義行のコードから、トークナイザーを使ってトークナイズした後、関数名部分を検出して
    mask_all=False の場合はその中からランダムに1トークンだけを [MASK] に置換し、
    mask_all=True の場合は関数名を構成する全トークンを [MASK] に置換します。

    入力例:
      {"func_code_string": "def my_function(x):\n    return x*2\n"}

    出力例（mask_all=False の場合）:
      {
        "func_code_string": "def [MASK] ( x ) :\n    return x*2\n",
        "target_function_name": "my_function"  # ただし、ここでは my_function の一部（ランダムに選んだ1トークン）が正解ラベルとなる
      }

    出力例（mask_all=True の場合）:
      {
        "func_code_string": "def [MASK] [MASK] ( x ) :\n    return x*2\n",
        "target_function_name": "my_function"  # 関数名全体が正解ラベル
      }

    Args:
        example (dict): 入力サンプル。キー "func_code_string" を含む。
        tokenizer (BertTokenizerFast): 事前に初期化されたトークナイザー
        mask_all (bool): True の場合、関数名を構成するすべてのトークンをマスクする。False の場合、関数名部分からランダムに1トークンのみをマスクする。

    Returns:
        dict: マスク後の "func_code_string" と、正解となる "target_function_name"（mask_all=False の場合は1トークン、mask_all=True の場合は全トークン）を含む。
    """
    code = example["func_code_string"]

    # コード全体をトークナイズ
    tokens = tokenizer.tokenize(code)

    # 最初のトークンが "def" であることを確認
    if len(tokens) < 3 or tokens[0].lower() != "def":
        example["target_function_name"] = None
        return example

    # 関数名部分は、"def" の直後から "(" が見つかるまでと仮定
    func_name_tokens = []
    i = 1
    while i < len(tokens) and "(" not in tokens[i]:
        func_name_tokens.append(tokens[i])
        i += 1

    if not func_name_tokens:
        example["target_function_name"] = None
        return example

    # マスク処理と正解ラベルの設定
    if mask_all:
        # 全トークンをマスク
        target_function_name = tokenizer.convert_tokens_to_string(func_name_tokens).strip()
        for j in range(1, 1 + len(func_name_tokens)):
            tokens[j] = "[MASK]"
    else:
        # ランダムに1トークンのみマスクする
        chosen_idx = random.randint(0, len(func_name_tokens) - 1)
        target_function_name = func_name_tokens[chosen_idx]
        # 関数名部分のトークンは位置 1 ～ (1+len(func_name_tokens)-1) に存在するので、
        # 選んだ位置は 1 + chosen_idx となる
        tokens[1 + chosen_idx] = "[MASK]"

    # マスク後のトークン列を文字列に戻す
    masked_text = tokenizer.convert_tokens_to_string(tokens)

    example["func_code_string"] = masked_text
    example["target_function_name"] = target_function_name
    return example


In [19]:
def func_tokenize_dataset_with_mask(dataset, tokenizer, max_length: int = 512):
    """
    CodeSearchNet のデータセット（すでに mask_function_name_with_tokenizer を適用済みのサンプル）
    をトークナイズします。

    ※ "func_code_string" をトークナイズし、padding と truncation を適用してシーケンス長を揃えた上で、
      正解ラベル "target_function_name" も保持します。
    ※ 最終的に "func_code_string"（生テキスト）のカラムは削除し、モデルが要求する数値データ（例：input_ids, attention_mask）と
      "target_function_name" のみを残します。

    Args:
        dataset (datasets.Dataset): 前処理済みのデータセット（各サンプルに "func_code_string" と "target_function_name" が含まれる）
        tokenizer (BertTokenizerFast): 学習済みトークナイザー
        max_length (int): トークナイズ時の最大シーケンス長

    Returns:
        datasets.Dataset: トークナイズ済みのデータセット（"input_ids", "attention_mask", "target_function_name" のみを含む）
    """
    def tokenize_function(example):
        # "func_code_string" をトークナイズし、padding と truncation を適用
        encoded = tokenizer(example["func_code_string"], truncation=True, padding="max_length", max_length=max_length)
        # 正解ラベルとして target_function_name を追加
        encoded["target_function_name"] = example["target_function_name"]
        return encoded

    # まず、トークナイズ処理を行う（入力となるカラムは "func_code_string" と "target_function_name"）
    tokenized_dataset = dataset.map(
        tokenize_function,
        batched=False,
        remove_columns=[col for col in dataset.column_names if col not in ["func_code_string", "target_function_name"]]
    )
    # 次に、不要な生テキストのカラム "func_code_string" を削除する
    tokenized_dataset = tokenized_dataset.remove_columns(["func_code_string"])
    return tokenized_dataset


# 例: raw_train_dataset, raw_eval_dataset は CodeSearchNet の元データセット
processed_eval_dataset  = raw_eval_dataset.map(lambda ex: mask_function_name_with_tokenizer(ex, tokenizer, mask_all=False))

tokenized_eval_dataset  = func_tokenize_dataset_with_mask(processed_eval_dataset, tokenizer, max_length=512)


Map:   0%|          | 0/23107 [00:00<?, ? examples/s]

Map:   0%|          | 0/23107 [00:00<?, ? examples/s]

In [20]:
def evaluate_function_name_prediction(eval_dataset, tokenizer, model, num_samples=100):
    """
    評価データセットからランダムに num_samples 件のサンプルについて、
    関数名部分（既に [MASK] に置換済み）の予測結果のトップ予測が
    target_function_name と一致しているかどうかを評価し、正解率を表示します。

    Args:
        eval_dataset (Dataset): 前処理済み評価用データセット（各サンプルに "func_code_string" と "target_function_name" が含まれていること）
        tokenizer (BertTokenizerFast): CodeBERT のトークナイザー
        model (BertForMaskedLM): CodeBERT のモデル
        num_samples (int): ランダムに評価するサンプル数（デフォルト: 100）
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    correct = 0
    total = 0
    errors = []

    # ランダムに num_samples 件のインデックスを取得
    sample_indices = random.sample(range(len(eval_dataset)), num_samples)

    for idx in sample_indices:
        sample = eval_dataset[idx]
        text = sample.get("func_code_string", "")
        target = sample.get("target_function_name", None)

        # target_function_name がないサンプルはスキップ
        if not target or target.strip() == "":
            continue

        # テキストをエンコード（最大512トークンに制限）
        token_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)

        # すでに mask_function_name 前処理により、関数名は [MASK] に置換されているはずです。
        # [MASK] トークンのIDが含まれているか確認
        if tokenizer.mask_token_id not in token_ids:
            # [MASK] が見つからなければサンプルをスキップ
            continue

        # [MASK] の最初の出現位置を取得
        mask_index = token_ids.index(tokenizer.mask_token_id)

        # テンソルに変換し、デバイスへ配置
        input_tensor = torch.tensor([token_ids], dtype=torch.long).to(device)

        model.eval()
        with torch.no_grad():
            outputs = model(input_tensor)
            logits = outputs.logits  # shape: (1, seq_len, vocab_size)

        # マスク位置のロジットから softmax を計算し、トップ予測を取得
        mask_logits = logits[0, mask_index]
        probs = F.softmax(mask_logits, dim=-1)
        top_idx = torch.argmax(probs).item()
        predicted_token = tokenizer.convert_ids_to_tokens(top_idx)

        total += 1
        if predicted_token.lower() == target.lower():
            correct += 1
        else:
            errors.append((text[:300], target, predicted_token))

    if total == 0:
        print("評価可能なサンプルが見つかりませんでした。")
    else:
        accuracy = correct / total * 100
        print(f"Function name prediction accuracy: {correct}/{total} = {accuracy:.2f}%")

        # エラー例を表示（必要に応じて）
        if errors:
            print("\n【エラー例】")
            for orig_text, tgt, pred in errors:
                print("元テキスト（最初の300文字）:", orig_text)
                print("正解関数名:", tgt)
                print("予測結果:", pred)
                print("-----")

# 例: 評価用データセット small_eval_dataset（前処理済み：mask_function_name など適用済み）を用いて評価
evaluate_function_name_prediction(processed_eval_dataset, tokenizer, model, num_samples=100)


Function name prediction accuracy: 54/97 = 55.67%

【エラー例】
元テキスト（最初の300文字）: def [MASK] ( self, a, b ) : " " " greater - than comparison " " " return operators. itebv ( 256, operators. ugt ( a, b ), 1, 0 )
正解関数名: gt
予測結果: cmp
-----
元テキスト（最初の300文字）: def _ [MASK] _ x ( self, word, suffix _ len ) : " " " return lovins ' condition x. parameters - - - - - - - - - - word : str word to check suffix _ len : int suffix length returns - - - - - - - bool true if condition is met " " " return word [ - suffix _ len - 1 ] in { ' i ', ' l ' } or ( word [ - s
正解関数名: cond
予測結果: condition
-----
元テキスト（最初の300文字）: def run _ [MASK] ( self, rs, overwrite = true, skip _ existing = false, path = none, chunksize = none ) : " " " compute timestamps for current populations. " " " if path is none : path = str ( self. s. store. filepath. parent ) kwargs = dict ( rs = rs, overwrite = overwrite, path = path, timeslice =
正解関数名: da
予測結果: populations
-----
元テキスト（最初の300文字）: def [MASK] _ preparing ( app ) : " " " wrap all s

In [22]:
import random
import torch
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizerFast
def evaluate_function_name_prediction(eval_dataset, tokenizer, model, num_samples=100):
    """
    評価データセットからランダムに num_samples 件のサンプルについて、
    関数名部分（既に [MASK] に置換済み）の予測結果のトップ予測が
    target_function_name と一致しているかどうかを評価し、正解率を表示します。

    Args:
        eval_dataset (Dataset): 前処理済み評価用データセット（各サンプルに "func_code_string" と "target_function_name" が含まれていること）
        tokenizer (BertTokenizerFast): CodeBERT のトークナイザー
        model (BertForMaskedLM): CodeBERT のモデル
        num_samples (int): ランダムに評価するサンプル数（デフォルト: 100）
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    correct = 0
    total = 0
    errors = []

    # ランダムに num_samples 件のインデックスを取得
    sample_indices = random.sample(range(len(eval_dataset)), num_samples)

    for idx in sample_indices:
        sample = eval_dataset[idx]
        text = sample.get("func_code_string", "")
        target = sample.get("target_function_name", None)

        # target_function_name がないサンプルはスキップ
        if not target or target.strip() == "":
            continue

        # テキストをエンコード（最大512トークンに制限）
        token_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)

        # すでに mask_function_name 前処理により、関数名は [MASK] に置換されているはずです。
        # [MASK] トークンのIDが含まれているか確認
        if tokenizer.mask_token_id not in token_ids:
            # [MASK] が見つからなければサンプルをスキップ
            continue

        # [MASK] の最初の出現位置を取得
        mask_index = token_ids.index(tokenizer.mask_token_id)

        # テンソルに変換し、デバイスへ配置
        input_tensor = torch.tensor([token_ids], dtype=torch.long).to(device)

        model.eval()
        with torch.no_grad():
            outputs = model(input_tensor)
            logits = outputs.logits  # shape: (1, seq_len, vocab_size)

        # マスク位置のロジットから softmax を計算し、トップ予測を取得
        mask_logits = logits[0, mask_index]
        probs = F.softmax(mask_logits, dim=-1)
        top_idx = torch.argmax(probs).item()
        predicted_token = tokenizer.convert_ids_to_tokens(top_idx)

        total += 1
        if predicted_token.lower() == target.lower():
            correct += 1
        else:
            errors.append((text[:300], target, predicted_token))

    if total == 0:
        print("評価可能なサンプルが見つかりませんでした。")
    else:
        accuracy = correct / total * 100
        print(f"Function name prediction accuracy: {correct}/{total} = {accuracy:.2f}%")

        # エラー例を表示（必要に応じて）
        if errors:
            print("\n【エラー例】")
            for orig_text, tgt, pred in errors:
                print("元テキスト（最初の300文字）:", orig_text)
                print("正解関数名:", tgt)
                print("予測結果:", pred)
                print("-----")

# 例: 評価用データセット small_eval_dataset（前処理済み：mask_function_name など適用済み）を用いて評価
evaluate_function_name_prediction(processed_eval_dataset, tokenizer, model, num_samples=3000)


Function name prediction accuracy: 1908/2961 = 64.44%

【エラー例】
元テキスト（最初の300文字）: def _ [MASK]rdate ( st ) : ' ' ' internal ' ' ' if isinstance ( st, string _ types ) : return st elif isinstance ( st, datetime ) : return st. strftime ( ' % y % m % d ' ) raise pyexception ( ' not a date : % s ', str ( st ) )
正解関数名: stro
予測結果: _
-----
元テキスト（最初の300文字）: def [MASK] _ skeleton ( self, filename, pid _ params = none ) : ' ' ' create and configure a skeleton in our model. parameters - - - - - - - - - - filename : str the name of a file containing skeleton configuration data. pid _ params : dict, optional if given, use this dictionary to set the pid cont
正解関数名: load
予測結果: create
-----
元テキスト（最初の300文字）: def _ [MASK] _ finder _ objectinfo ( objectinfo, varinfo, findercmap, finderconvolve, sigclip, normto, normmingap, deredden _ object = true, custom _ bandpasses = none, lclistpkl = none, nbrradiusarcsec = 30. 0, maxnumneighbors = 5, plotdpi = 100, findercachedir = ' ~ /. astrobase / stamp - cache ',
正

In [23]:
import random
import torch
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizerFast

def evaluate_function_name_prediction(eval_dataset, tokenizer, model, num_samples=100):
    """
    評価データセットからランダムに num_samples 件のサンプルについて、
    関数名部分（既に [MASK] に置換済み）の予測結果のトップ予測が
    target_function_name と一致しているかどうか（または正答の一部を含むか）を評価し、
    正解率を表示します。

    Args:
        eval_dataset (Dataset): 前処理済み評価用データセット（各サンプルに "func_code_string" と "target_function_name" が含まれていること）
        tokenizer (BertTokenizerFast): CodeBERT のトークナイザー
        model (BertForMaskedLM): CodeBERT のモデル
        num_samples (int): ランダムに評価するサンプル数（デフォルト: 100）
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    correct = 0
    total = 0
    errors = []

    # ランダムに num_samples 件のインデックスを取得
    sample_indices = random.sample(range(len(eval_dataset)), num_samples)

    for idx in sample_indices:
        sample = eval_dataset[idx]
        text = sample.get("func_code_string", "")
        target = sample.get("target_function_name", None)

        # target_function_name がないサンプルはスキップ
        if not target or target.strip() == "":
            continue

        # テキストをエンコード（最大512トークンに制限）
        token_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)

        # すでに mask_function_name 前処理により、関数名は [MASK] に置換されているはずです。
        # [MASK] トークンのIDが含まれているか確認
        if tokenizer.mask_token_id not in token_ids:
            # [MASK] が見つからなければサンプルをスキップ
            continue

        # [MASK] の最初の出現位置を取得
        mask_index = token_ids.index(tokenizer.mask_token_id)

        # テンソルに変換し、デバイスへ配置
        input_tensor = torch.tensor([token_ids], dtype=torch.long).to(device)

        model.eval()
        with torch.no_grad():
            outputs = model(input_tensor)
            logits = outputs.logits  # shape: (1, seq_len, vocab_size)

        # マスク位置のロジットから softmax を計算し、トップ予測を取得
        mask_logits = logits[0, mask_index]
        probs = F.softmax(mask_logits, dim=-1)
        top_idx = torch.argmax(probs).item()
        predicted_token = tokenizer.convert_ids_to_tokens(top_idx)

        total += 1
        # 予測トークンが正解の文字列の一部に含まれていれば正解とする
        if predicted_token.lower() in target.lower():
            correct += 1
        else:
            errors.append((text[:300], target, predicted_token))

    if total == 0:
        print("評価可能なサンプルが見つかりませんでした。")
    else:
        accuracy = correct / total * 100
        print(f"Function name prediction accuracy: {correct}/{total} = {accuracy:.2f}%")

processed_all_eval_dataset  = raw_eval_dataset.map(lambda ex: mask_function_name_with_tokenizer(ex, tokenizer, mask_all=all))
# 例: 評価用データセット processed_eval_dataset（前処理済み：mask_function_name など適用済み）を用いて評価
evaluate_function_name_prediction(processed_all_eval_dataset, tokenizer, model, num_samples=23107)

Map:   0%|          | 0/23107 [00:00<?, ? examples/s]

Function name prediction accuracy: 15148/22873 = 66.23%


In [24]:
import random
import torch
import torch.nn.functional as F
from transformers import BertForMaskedLM, BertTokenizerFast

def evaluate_function_name_prediction(eval_dataset, tokenizer, model, num_samples=100):
    """
    評価データセットからランダムに num_samples 件のサンプルについて、
    関数名部分（既に [MASK] に置換済み）の各 [MASK] トークンに対して予測を行い、
    その予測結果（複数トークンの場合は結合した文字列）が target_function_name と一致しているかどうかを評価し、
    正解率を表示します。

    Args:
        eval_dataset (Dataset): 前処理済み評価用データセット（各サンプルに "func_code_string" と "target_function_name" が含まれていること）
        tokenizer (BertTokenizerFast): CodeBERT のトークナイザー
        model (BertForMaskedLM): CodeBERT のモデル
        num_samples (int): ランダムに評価するサンプル数（デフォルト: 100）
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    correct = 0
    total = 0
    errors = []

    # ランダムに num_samples 件のサンプルを選択
    sample_indices = random.sample(range(len(eval_dataset)), num_samples)

    for idx in sample_indices:
        sample = eval_dataset[idx]
        text = sample.get("func_code_string", "")
        target = sample.get("target_function_name", None)

        # target_function_name がないサンプルはスキップ
        if not target or target.strip() == "":
            continue

        # テキストをエンコード（最大512トークンに制限）
        token_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True)

        # テキスト内に含まれるすべての [MASK] トークンのインデックスを取得
        mask_indices = [i for i, token_id in enumerate(token_ids) if token_id == tokenizer.mask_token_id]
        if not mask_indices:
            # [MASK] トークンが見つからなければスキップ
            continue

        # 入力テンソルに変換し、デバイスへ配置
        input_tensor = torch.tensor([token_ids], dtype=torch.long).to(device)

        model.eval()
        with torch.no_grad():
            outputs = model(input_tensor)
            logits = outputs.logits  # shape: (1, seq_len, vocab_size)

        # 各 [MASK] トークン位置についてトップの予測を取得
        predicted_tokens = []
        for mask_index in mask_indices:
            mask_logits = logits[0, mask_index]
            top_idx = torch.argmax(mask_logits).item()
            token = tokenizer.convert_ids_to_tokens(top_idx)
            predicted_tokens.append(token)

        # 取得した各トークンを結合して予測された関数名とする
        # tokenizer.convert_tokens_to_string は、WordPiece の "##" などを考慮して結合してくれます。
        predicted_function_name = tokenizer.convert_tokens_to_string(predicted_tokens).strip()

        total += 1
        if predicted_function_name.lower() == target.lower():
            correct += 1
        else:
            errors.append((text[:300], target, predicted_function_name))

    if total == 0:
        print("評価可能なサンプルが見つかりませんでした。")
    else:
        accuracy = correct / total * 100
        print(f"Function name prediction accuracy: {correct}/{total} = {accuracy:.2f}%")


# 例: 評価用データセット processed_eval_dataset（前処理済み：mask_function_name など適用済み）を用いて評価
evaluate_function_name_prediction(processed_all_eval_dataset, tokenizer, model, num_samples=23107)


Function name prediction accuracy: 6304/22873 = 27.56%
